# Getting "context chunks" and topic modeling those chunks


If we have the start or end characters, Let's you use those start or end characters to get some of the original text surrounding that named entity.

+ We're going to get 400 character chunks around our quotations. 
+ Then we'll get  most frequent non-stopword words in our chunks
+ Then, we'll also run topic models on those chunks of context


## Import

In [ ]:
!pip3 install --user text-matcher

In [ ]:
from text_matcher.matcher import Text, Matcher
import json
import nltk
nltk.download('stopwords')
import pandas as pd
from IPython.display import clear_output
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 6]
#pd.set_option('display.max_colwidth', None)

## Define filepath

In [3]:
# Load Middlemarch .txt file 
# (Note: must have 'middlemarch.txt' in this directory)
with open('middlemarch.txt') as f: 
    rawMM = f.read()

mm = Text(rawMM, 'Middlemarch')

# Load in the JSON file with our JSTOR articles and data from TextMatcher
# (Note: must have the file 'default.json' in the same directory as this notebook)
df = pd.read_json('hyperparameter-data/t2-c3-n2-m3-no-stops.json')

## Let's set the quote start and end characters in Middlemarch and verify
In our notes, we have the start and end characters for the passage in Middlemarch. Let's confirm it

In [33]:
# Let's check the start/end characters in MM for the squirrel's heartbeat
# Squirrel's heartbeat
mm.text[411903:412109]
# Chapter 21 (in 2010s) -in 2010s - moral stupidity
#mm.text[449667:449764]
# Chapter 15 (in 1980s and 1990s - unraveling-certain-human-lots)
#mm.text[291674:291941]
# Chapter 15 (in 1980s and 1990s - great Frenchman)
#mm.text[307181:307546]
# Chapter 15 (in 1980s and 1990s - great seer)
#mm.text[308560:308896]
# Chapter 15 all
#mm.text[290371:322052]
# Finale (many-who-knew-her)
#mm.text[1787696:1787884]
# Finale (last-sentence-full-VS-journals)
mm.text[1793134:1793448]
# Finale (last-sentence-partial-VS-journals)
#mm.text[1793210:1793448] 

'But the effect of her being on those around her was\nincalculably diffusive: for the growing good of the world is partly\ndependent on unhistoric acts; and that things are not so ill with you\nand me as they might have been, is half owing to the number who lived\nfaithfully a hidden life, and rest in unvisited tombs.'

In [34]:
quote_index = [1793134, 1793448] 

In [30]:
# name_of_passage = "Chapter21-moral-stupidity"
#name_of_passage = "Chapter15-unraveling-certain-human-lots-80s-90s"
#name_of_passage = "Chapter20-squirrels-heartbeat"
#name_of_passage = "Chapter15-ALL"
#name_of_passage = "Finale-last-sentence-full-VS-journals"
name_of_passage = "All-Victorianist-journals"

In [5]:
# Let's look just at just our character indexes for quotations
df = df[['id', 'Locations in A', 'Locations in B']]
df

,id,Locations in A,Locations in B
0,http://www.jstor.org/stable/41158244,"[[130022, 130046]]","[[6851, 6875]]"
1,http://www.jstor.org/stable/3722819,[],[]
2,http://www.jstor.org/stable/40137021,[],[]
3,http://www.jstor.org/stable/802346,[],[]
4,http://www.jstor.org/stable/10.5325/style.45.4...,[],[]
...,...,...,...
5879,http://www.jstor.org/stable/27747183,"[[23799, 24121]]","[[41472, 41793]]"
5880,http://www.jstor.org/stable/4123220,[],[]
5881,http://www.jstor.org/stable/24575734,"[[173657, 173756]]","[[33963, 34061]]"
5882,http://www.jstor.org/stable/3827815,[],[]


In [6]:
# Loop over DF to get ID and each individidual match in Locations in B and each in B
df = df.explode(['Locations in A', 'Locations in B'])


In [7]:
# Let's get just the matches that are non-null
df_just_quotes = df[df['Locations in A'].notna()]
df_just_quotes

,id,Locations in A,Locations in B
0,http://www.jstor.org/stable/41158244,"[130022, 130046]","[6851, 6875]"
9,http://www.jstor.org/stable/2168205,"[598742, 598885]","[71966, 72109]"
17,http://www.jstor.org/stable/2933224,"[24114, 24349]","[3325, 3559]"
17,http://www.jstor.org/stable/2933224,"[25387, 25435]","[5318, 5366]"
17,http://www.jstor.org/stable/2933224,"[39572, 39677]","[16007, 16112]"
...,...,...,...
5862,http://www.jstor.org/stable/41349043,"[1292109, 1292133]","[88992, 89016]"
5863,http://www.jstor.org/stable/24780708,"[734162, 734292]","[43955, 44085]"
5870,http://www.jstor.org/stable/25293438,"[1013, 1149]","[40346, 40482]"
5879,http://www.jstor.org/stable/27747183,"[23799, 24121]","[41472, 41793]"


### Let's get all the ids, Locations in B and Locations in A for just our specified character index range (eg, squirrel's heartbeat

In [39]:
# Define a function that can compare two intervals in the form of listed tuples, eg [x0, x1], [y0,y1]
def overlaps(range1, range2):
    try:
        if len(range1) == 2 and len(range2) == 2:
            return max(range1[0],range2[0]) < min(range1[1],range2[1])
        else:
            return False
    except TypeError:
        return False

In [40]:
quotation_locations_in_A = []
quotation_locations_in_B = []
ids = []


# Let's extract just the Locations in A that overlap with the squirrel's hearbeat quotation
for item in range(len(df_just_quotes)):
    loc_in_a_index = df_just_quotes['Locations in A'].iloc[item]
    if overlaps(quote_index, loc_in_a_index) == True:
        quotation_locations_in_A.append(df_just_quotes['Locations in A'].iloc[item])
        quotation_locations_in_B.append(df_just_quotes['Locations in B'].iloc[item])
        ids.append(df_just_quotes['id'].iloc[item])

In [13]:
# Alternate for VS
quotation_locations_in_A = []
quotation_locations_in_B = []
ids = []

for item in range(len(df_just_quotes)):
    loc_in_a_index = df_just_quotes['Locations in A'].iloc[item]
    quotation_locations_in_A.append(df_just_quotes['Locations in A'].iloc[item])
    quotation_locations_in_B.append(df_just_quotes['Locations in B'].iloc[item])
    ids.append(df_just_quotes['id'].iloc[item])

In [14]:
zipped = list(zip(ids, quotation_locations_in_A, quotation_locations_in_B))
just_matches_df = pd.DataFrame(zipped, columns=['id', 'Locations in A', 'Locations in B'])
just_matches_df

,id,Locations in A,Locations in B
0,http://www.jstor.org/stable/41158244,"[130022, 130046]","[6851, 6875]"
1,http://www.jstor.org/stable/2168205,"[598742, 598885]","[71966, 72109]"
2,http://www.jstor.org/stable/2933224,"[24114, 24349]","[3325, 3559]"
3,http://www.jstor.org/stable/2933224,"[25387, 25435]","[5318, 5366]"
4,http://www.jstor.org/stable/2933224,"[39572, 39677]","[16007, 16112]"
...,...,...,...
3795,http://www.jstor.org/stable/41349043,"[1292109, 1292133]","[88992, 89016]"
3796,http://www.jstor.org/stable/24780708,"[734162, 734292]","[43955, 44085]"
3797,http://www.jstor.org/stable/25293438,"[1013, 1149]","[40346, 40482]"
3798,http://www.jstor.org/stable/27747183,"[23799, 24121]","[41472, 41793]"


### Let's use the start and end characters to extract a context chunk left & right of our quotations

Set the `num_characters_before_quote` and `num_characters_after_quote`

In [15]:
# Use article_id to get the index of the article in our DataFrame
df_with_fulltexts = pd.read_json('default.json')
df_with_fulltexts['year'] = pd.DatetimeIndex(df_with_fulltexts['datePublished']).year
df_with_fulltexts['Decade'] = df_with_fulltexts['year'] - (df_with_fulltexts['year'] % 10)

In [31]:
# Change these to desired chunk
num_characters_before_quote = 200
num_characters_after_quote = 750
decades_to_check = [2010, 2000, 1990, 1980, 1970, 1960, 1950]
list_of_VS_journals = ['Victorian Studies', 'George Eliot - George Henry Lewes Studies', 'Nineteenth-Century Fiction', 'Nineteenth-Century Literature', 'Dickens Studies Annual', 'Victorian Literature and Culture', 'Victorian Review', 'The George Eliot, George Henry Lewes Newsletter', 'Victorian Periodicals Review', 'Dickens Quarterly', 'Victorian Poetry', 'The Thomas Hardy Journal', 'The Gaskell Society Journal', 'The Gaskell Journal', 'Newsletter of the Victorian Studies Association of Western Canada', 'Dickens Studies Newsletter', 'Browning Institute Studies', 'Victorian Periodicals Newsletter', 'Carlyle Studies Annual', 'Conradiana', 'Tennyson Research Bulletin', 'The Conradian', 'The Hardy Society Journal', 'The Hardy Review', 'Studies in Browning and His Circle', 'Nineteenth-Century French Studies', 'The Wilkie Collins Journal', 'Carlyle Newsletter', 'The Wildean', 'Dickens Studies', 'Carlyle Annual', '19th-Century Music', 'The Trollopian', 'Conrad Studies']

# Create an empty list that we will populate with the contexts for quotations
context_chunks_for_quotations = []
context_chunks_ids = []
context_chunks_decade = []


# Loop over each of the start and end locations to produce a x-character chunk of context
for item in range(len(just_matches_df)):
    article_URL = just_matches_df['id'].loc[item]
    startandEndLocations = just_matches_df['Locations in B'].loc[item]
    print(article_URL)
    print(startandEndLocations[0])
    print(startandEndLocations[1])
    article_index = df_with_fulltexts[df_with_fulltexts['id'] == article_URL].index[0]
    print(article_index)
    article_title = df_with_fulltexts['title'].loc[article_index]
    print(article_title)
    article_decade = df_with_fulltexts['Decade'].loc[article_index]
    print(article_decade)
    article_journal = df_with_fulltexts['isPartOf'].loc[article_index]
    article_text = df_with_fulltexts['fullText'].loc[article_index]
    cleaned_article_text = Text(article_text, article_title)
    if article_journal in list_of_VS_journals:
    #if article_decade in decades_to_check:
        context_chunks_for_quotations.append((cleaned_article_text.text[startandEndLocations[0]-num_characters_before_quote:startandEndLocations[0]]) + "[...]" + (cleaned_article_text.text[startandEndLocations[1]:startandEndLocations[1]+num_characters_after_quote]))
        context_chunks_ids.append(df_with_fulltexts['id'].loc[article_index])
        context_chunks_decade.append(df_with_fulltexts['Decade'].loc[article_index])

http://www.jstor.org/stable/41158244
6851
6875
0
Review Article
2000
http://www.jstor.org/stable/2168205
71966
72109
9
Gender and the Practices of Scientific History: The Seminar and Archival Research in the Nineteenth Century
1990
http://www.jstor.org/stable/2933224
3325
3559
17
Plexuses and Ganglia: Scientific Allusion in Middlemarch
1970
http://www.jstor.org/stable/2933224
5318
5366
17
Plexuses and Ganglia: Scientific Allusion in Middlemarch
1970
http://www.jstor.org/stable/2933224
16007
16112
17
Plexuses and Ganglia: Scientific Allusion in Middlemarch
1970
http://www.jstor.org/stable/2933224
27375
27414
17
Plexuses and Ganglia: Scientific Allusion in Middlemarch
1970
http://www.jstor.org/stable/2933224
27415
27608
17
Plexuses and Ganglia: Scientific Allusion in Middlemarch
1970
http://www.jstor.org/stable/2933224
27663
27876
17
Plexuses and Ganglia: Scientific Allusion in Middlemarch
1970
http://www.jstor.org/stable/2933224
27883
28010
17
Plexuses and Ganglia: Scientific Allusion i

http://www.jstor.org/stable/43282359
36321
36358
105
Standing in the Last Ditch: On the Communicative Intentions of Fiction Makers
2010
http://www.jstor.org/stable/42827960
73073
73093
107
GEORGE ELIOT'S READING: A CHRONOLOGICAL LIST
2000
http://www.jstor.org/stable/10.5325/georelioghlstud.70.2.0143
350
876
108
Hidden Allusion in the Finale of <em>Middlemarch</em>: George Eliot and the Jewish Myth of the <em>Lamed Vov</em>
2010
http://www.jstor.org/stable/450490
22486
22511
113
"The Lifted Veil" and George Eliot's Early Aesthetic
1980
http://www.jstor.org/stable/450490
41261
41282
113
"The Lifted Veil" and George Eliot's Early Aesthetic
1980
http://www.jstor.org/stable/450490
48292
48339
113
"The Lifted Veil" and George Eliot's Early Aesthetic
1980
http://www.jstor.org/stable/450490
48347
48388
113
"The Lifted Veil" and George Eliot's Early Aesthetic
1980
http://www.jstor.org/stable/24542541
26530
26570
122
In Defense of Paraphrase
2010
http://www.jstor.org/stable/24542541
26597
27290


http://www.jstor.org/stable/27710458
35046
35105
171
"Middlemarch" and George Eliot's Female (Re) Vision of Shakespeare
1990
http://www.jstor.org/stable/44372200
27017
27114
172
Dickens and Gender: Recent Studies, 1992-2007
2000
http://www.jstor.org/stable/10.1525/ncl.2011.66.3.307
1517
1650
173
The Rosamond Plots
2010
http://www.jstor.org/stable/10.1525/ncl.2011.66.3.307
29069
29141
173
The Rosamond Plots
2010
http://www.jstor.org/stable/10.1525/ncl.2011.66.3.307
31051
31223
173
The Rosamond Plots
2010
http://www.jstor.org/stable/10.1525/ncl.2011.66.3.307
34087
34175
173
The Rosamond Plots
2010
http://www.jstor.org/stable/10.1525/ncl.2011.66.3.307
34392
34443
173
The Rosamond Plots
2010
http://www.jstor.org/stable/10.1525/ncl.2011.66.3.307
34516
34584
173
The Rosamond Plots
2010
http://www.jstor.org/stable/10.1525/ncl.2011.66.3.307
34777
34818
173
The Rosamond Plots
2010
http://www.jstor.org/stable/10.1525/ncl.2011.66.3.307
34949
35087
173
The Rosamond Plots
2010
http://www.jstor.org/

237
The Structure of Dickens's Imagination in Little Dorrit
1980
http://www.jstor.org/stable/436740
20373
20403
241
Affective Coherence, a Principle of Abated Action, and Meredith's "Modern Love"
1970
http://www.jstor.org/stable/1556298
106
153
255
George Eliot's Realism and Adam Smith
2000
http://www.jstor.org/stable/1556298
195
246
255
George Eliot's Realism and Adam Smith
2000
http://www.jstor.org/stable/1556298
16529
16846
255
George Eliot's Realism and Adam Smith
2000
http://www.jstor.org/stable/1556298
34358
34635
255
George Eliot's Realism and Adam Smith
2000
http://www.jstor.org/stable/1345698
5386
5653
258
George Eliot and the Production of Consumers
1990
http://www.jstor.org/stable/1345698
5659
5725
258
George Eliot and the Production of Consumers
1990
http://www.jstor.org/stable/1345698
6663
6976
258
George Eliot and the Production of Consumers
1990
http://www.jstor.org/stable/1345698
9772
9807
258
George Eliot and the Production of Consumers
1990
http://www.jstor.org/stable

http://www.jstor.org/stable/44235888
2332
2375
391
Review Article
2010
http://www.jstor.org/stable/43595508
19642
19689
393
SILAS MARNER—GEORGE ELIOT'S MALE HEROINE
1990
http://www.jstor.org/stable/20084157
25671
25798
410
Periodical Partners: A Context for Teaching Victorian Literature and Science
2000
http://www.jstor.org/stable/461001
49311
49354
411
Wuthering Heights and the Limits of Vision
1970
http://www.jstor.org/stable/10.5325/style.45.4.654
10245
10334
416
Kinds of Minds: On Alan Palmer's “Social Minds”
2010
http://www.jstor.org/stable/3484831
14114
14344
423
William Graham Sumner: An Essay in the Sociology of Knowledge
1950
http://www.jstor.org/stable/4544196
4693
4922
425
Greatness and Goodness: The Presidential Address
1930
http://www.jstor.org/stable/26377185
38367
38560
428
“A True Prophet”? Speculation in Victorian Sensory Physiology and George Eliot’s “The Lifted Veil”
2010
http://www.jstor.org/stable/45374127
1264
1691
429
GEORGE ELIOT AND LINDLEY MURRAY
1980
http://w

...

http://www.jstor.org/stable/4289104
22323
22383
5404
Reading the Signs
1990
http://www.jstor.org/stable/41435202
2288
2349
5406
COMMONPLACE BOOK: Privacy
2010
http://www.jstor.org/stable/41435202
2363
2632
5406
COMMONPLACE BOOK: Privacy
2010
http://www.jstor.org/stable/450565
33558
33768
5408
Astrophil and Estella: A Defense of Poesy
1990
http://www.jstor.org/stable/3826242
17031
17463
5410
George Eliot, Positivism, and the Social Vision of "Middlemarch"
1970
http://www.jstor.org/stable/3826242
17342
17509
5410
George Eliot, Positivism, and the Social Vision of "Middlemarch"
1970
http://www.jstor.org/stable/3826242
17515
17547
5410
George Eliot, Positivism, and the Social Vision of "Middlemarch"
1970
http://www.jstor.org/stable/3826242
17924
17959
5410
George Eliot, Positivism, and the Social Vision of "Middlemarch"
1970
http://www.jstor.org/stable/3826242
39639
39749
5410
George Eliot, Positivism, and the Social Vision of "Middlemarch"
1970
http://www.jstor.org/stable/3826242
39788
39

http://www.jstor.org/stable/2932437
16978
17085
5476
George Eliot's Unascribed Mottoes
1960
http://www.jstor.org/stable/2932437
17090
17204
5476
George Eliot's Unascribed Mottoes
1960
http://www.jstor.org/stable/3044869
23623
23647
5479
Questions of Method: Some Unpublished Late Essays
1980
http://www.jstor.org/stable/2933193
3640
3712
5481
Review Article
1970
http://www.jstor.org/stable/44315070
3793
4031
5482
The Way George Eliot Read "Paradise Lost": Raphael and the Temptation of Speculative Thought
2010
http://www.jstor.org/stable/44315070
6123
6439
5482
The Way George Eliot Read "Paradise Lost": Raphael and the Temptation of Speculative Thought
2010
http://www.jstor.org/stable/44315070
6341
6458
5482
The Way George Eliot Read "Paradise Lost": Raphael and the Temptation of Speculative Thought
2010
http://www.jstor.org/stable/44315070
6657
6878
5482
The Way George Eliot Read "Paradise Lost": Raphael and the Temptation of Speculative Thought
2010
http://www.jstor.org/stable/44315070


http://www.jstor.org/stable/460176
42719
42846
5559
Visions and Revisions: Chapter LXXXI of Middlemarch
1950
http://www.jstor.org/stable/460176
42867
43064
5559
Visions and Revisions: Chapter LXXXI of Middlemarch
1950
http://www.jstor.org/stable/460176
44297
44829
5559
Visions and Revisions: Chapter LXXXI of Middlemarch
1950
http://www.jstor.org/stable/460176
44735
45040
5559
Visions and Revisions: Chapter LXXXI of Middlemarch
1950
http://www.jstor.org/stable/2933652
9934
9961
5564
Struggling for Medical Reform in Middlemarch
1990
http://www.jstor.org/stable/2933652
9963
10104
5564
Struggling for Medical Reform in Middlemarch
1990
http://www.jstor.org/stable/2933652
11659
11683
5564
Struggling for Medical Reform in Middlemarch
1990
http://www.jstor.org/stable/2933652
11859
12011
5564
Struggling for Medical Reform in Middlemarch
1990
http://www.jstor.org/stable/2933652
25866
25902
5564
Struggling for Medical Reform in Middlemarch
1990
http://www.jstor.org/stable/2933652
25921
26003
5564

http://www.jstor.org/stable/10.1525/ncl.2010.65.2.214
33340
33686
5636
"This Immense Expense of Art": George Eliot and John Ruskin on Consumption and the Limits of Sympathy
2010
http://www.jstor.org/stable/10.1525/ncl.2010.65.2.214
40324
40495
5636
"This Immense Expense of Art": George Eliot and John Ruskin on Consumption and the Limits of Sympathy
2010
http://www.jstor.org/stable/10.1525/ncl.2010.65.2.214
40502
40577
5636
"This Immense Expense of Art": George Eliot and John Ruskin on Consumption and the Limits of Sympathy
2010
http://www.jstor.org/stable/10.1525/ncl.2010.65.2.214
78110
78145
5636
"This Immense Expense of Art": George Eliot and John Ruskin on Consumption and the Limits of Sympathy
2010
http://www.jstor.org/stable/24543363
9811
9834
5638
FEMALE MATE-GUARDING IN LAWRENCE'S "WINTRY PEACOCK": AN EVOLUTIONARY PERSPECTIVE
2010
http://www.jstor.org/stable/24464620
2334
2869
5640
The Portrait in the Spoon: George Eliot's Casaubon and John Milton
1980
http://www.jstor.org/stabl

http://www.jstor.org/stable/30225027
14917
15035
5816
Vraisemblance and Alienation Techniques: The Basis for Reflexivity in Fiction
1980
http://www.jstor.org/stable/30225027
15082
15153
5816
Vraisemblance and Alienation Techniques: The Basis for Reflexivity in Fiction
1980
http://www.jstor.org/stable/30225027
16210
16278
5816
Vraisemblance and Alienation Techniques: The Basis for Reflexivity in Fiction
1980
http://www.jstor.org/stable/30225027
18850
18966
5816
Vraisemblance and Alienation Techniques: The Basis for Reflexivity in Fiction
1980
http://www.jstor.org/stable/30225027
19042
19075
5816
Vraisemblance and Alienation Techniques: The Basis for Reflexivity in Fiction
1980
http://www.jstor.org/stable/26442711
69654
69685
5820
NOTES ON CURRENT BOOKS
1960
http://www.jstor.org/stable/j.ctt22727pw.10
71919
72043
5827
Quo Vadis:
2010
http://www.jstor.org/stable/27793616
8770
8812
5832
Interdisciplinary or Merely Undisciplined? The Teaching/Research Nexus
2000
http://www.jstor.org/stable/

### Print the contexts around quotations:

In [ ]:
context_chunks_for_quotations

In [52]:
context_chunks_ids

['http://www.jstor.org/stable/10.5325/georelioghlstud.70.2.0143',
 'http://www.jstor.org/stable/3828714',
 'http://www.jstor.org/stable/10.1525/ncl.2012.67.2.204',
 'http://www.jstor.org/stable/45290578',
 'http://www.jstor.org/stable/10.5325/georelioghlstud.69.2.0160',
 'http://www.jstor.org/stable/10.5325/georelioghlstud.67.1.0056',
 'http://www.jstor.org/stable/42827730',
 'http://www.jstor.org/stable/3044570',
 'http://www.jstor.org/stable/41039114',
 'http://www.jstor.org/stable/42827891',
 'http://www.jstor.org/stable/26384549',
 'http://www.jstor.org/stable/26584943',
 'http://www.jstor.org/stable/26503668']

### Print the context around the fifth quotation

In [32]:
print(context_chunks_for_quotations[3])

l, in a passage that begins playfully but concludes as a paradigm for the modern novelist, George Eliot demonstrates the inadequacy of telescopic sight and its necessary replacement by microscopic: a [...] without witnessing any interview that could excite suspicion, or any scene from which [Mrs. 
 44 Nineteenth-Century Fiction Cadwallader] did not return with the same unperturbed keenness of eye.... Even with a microscope directed on a waterdrop we find ourselves making interpretations which turn out to be rather coarse; for whereas under a weak lens you may seem to see a creature exhibiting an active voracity into which other smaller creatures actively play ... a stronger lens reveals to you certain tiniest hairlets which make vortices for these victims while the swallower waits passively. . . . In this way, metaphorically speaking, a strong lens applied to [her] match-making will show a play of minute causes producing . . . thought and sp


## To write output to a single text file

In [36]:
name_of_quote = f"{name_of_passage}-{num_characters_before_quote}-char-before-and-{num_characters_after_quote}-after"
output_file = open(f'VS-and-Victorian-studies/quotation-contexts-{name_of_quote}.txt', mode='w', encoding='utf-8')

for context in context_chunks_for_quotations:
     output_file.write(context)
     output_file.write('\n')
output_file.close()

In [661]:
#name_of_quote = f"{name_of_passage}-fulltexts"

## To write both contexts and IDs to a CSV


In [48]:
zipped = list(zip(context_chunks_ids, context_chunks_decade, context_chunks_for_quotations))
chunks_with_ids_df = pd.DataFrame(zipped, columns=['id', 'Decade', 'Quotation Context Chunk'])
chunks_with_ids_df.to_csv(f'quotation-contexts-ids-{name_of_quote}.csv', encoding ='utf-8')

## To write output to multiple text files in a new directory

In [49]:
# To output our context for quotations as a series of new files with the same beginning, followed by the number of the section

#Import pathlib 
from pathlib import Path

# Define and name the new output directory using pathlib
path = Path(f'{name_of_quote}_contexts/')
path.mkdir(exist_ok=True)

# Set the prefix for our output files, followed by the number of the section
begining_of_output_filenames = f'{name_of_quote}_contexts-'

# Iterate over each of the chunks of context for BookNLP NER
for i in range(1, len(context_chunks_for_quotations)+1):
    open(str(path) + "/" + begining_of_output_filenames+str(i)+'.txt','w').write(context_chunks_for_quotations[i-1])

## To look at most frequent words

In [37]:
import re
from collections import Counter

# Define Functions

def split_into_words(any_chunk_of_text):
    lowercase_text = any_chunk_of_text.lower()
    split_words = re.split("\W+", lowercase_text) 
    return split_words

# Define Filepaths and Assign Variables

#directory_path = f'{name_of_quote}_contexts/'
filename = f'VS-and-Victorian-studies/quotation-contexts-{name_of_quote}.txt'
number_of_desired_words = 100

stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 've', 'll', 'amp',
]

most_frequent_non_stopwords = []


full_text = open(filename, encoding="utf-8", errors='ignore').read()
all_the_words = split_into_words(full_text)
non_stopwords = [word for word in all_the_words if word not in stopwords]
non_stopwords_tally = Counter(non_stopwords)
most_frequent_non_stopwords_to_add = non_stopwords_tally.most_common(number_of_desired_words)
most_frequent_non_stopwords.append(most_frequent_non_stopwords_to_add)

# Output Results

most_frequent_non_stopwords


[[('dorothea', 1314),
  ('eliot', 980),
  ('middlemarch', 748),
  ('lydgate', 616),
  ('p', 594),
  ('george', 551),
  ('casaubon', 535),
  ('one', 470),
  ('rosamond', 399),
  ('ch', 396),
  ('novel', 368),
  ('mr', 360),
  ('life', 351),
  ('like', 346),
  ('would', 323),
  ('ladislaw', 299),
  ('brooke', 283),
  ('chapter', 278),
  ('narrator', 274),
  ('also', 272),
  ('even', 251),
  ('see', 249),
  ('man', 245),
  ('first', 236),
  ('women', 220),
  ('world', 217),
  ('new', 213),
  ('could', 202),
  ('marriage', 202),
  ('e', 194),
  ('mrs', 193),
  ('woman', 191),
  ('may', 189),
  ('time', 185),
  ('much', 181),
  ('young', 180),
  ('self', 180),
  ('1', 179),
  ('social', 177),
  ('us', 175),
  ('work', 165),
  ('must', 164),
  ('though', 164),
  ('rather', 163),
  ('victorian', 162),
  ('well', 161),
  ('might', 161),
  ('century', 159),
  ('husband', 155),
  ('good', 155),
  ('little', 155),
  ('yet', 153),
  ('way', 152),
  ('bulstrode', 151),
  ('great', 146),
  ('another

In [35]:
output_file = open(f'VS-and-Victorian-studies/quotation-contexts-most-frequent-words-{name_of_quote}.txt', mode='w', encoding='utf-8')

for item in most_frequent_non_stopwords:
    for word in item:
        print(word[0])
        output_file.write(word[0])
        output_file.write('\n')
output_file.close()

dorothea
eliot
middlemarch
lydgate
p
george
casaubon
one
rosamond
ch
novel
mr
life
like
would
ladislaw
brooke
chapter
narrator
also
even
see
man
first
women
world
new
could
marriage
e
mrs
woman
may
time
much
young
self
1
social
us
work
must
though
rather
victorian
well
might
century
husband
good
little
yet
way
bulstrode
great
another
two
however
made
love
make
knowledge
view
whose
mary
reader
nineteenth
later
know
things
fred
mind
sense
characters
point
end
never
human
character
many
nature
thought
seems
pp
scene
n
without
come
passage
others
men
light
r
political
people
says
history
naumann
fiction
d


## To Topic Model our  contents

In [312]:
#!pip install little_mallet_wrapper

In [55]:
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path

## NOTE: we have to install openjdk + mallet
To install open Java Development Kit, either download from the 

## Step 1: Download and Install Java Development Kit

Go to the Java Development Kit download page, find your operating system, and click on the corresponding download link: https://www.oracle.com/java/technologies/javase-jdk14-downloads.html

Choose the most recent JDK (currently JDK 16) and click "JDK Download." This will take you to an installation page. Choose the appropriate file to download: 

- For Mac: macOS Installer (ARM is for Macs with M1/M2 chips, x64 is for Macs with Intel processors)
- For Windows:  Windowsx64 Installer
- For Linux: Windowsx64 Installer

Then open or unzip the file and follow all the instructions to complete the installation.

## Step 2: Tell your computer where to find Java

Now we have to tell our computers where to find JDK. For Macs, this involves defining a new variable, called JAVA_HOME and give it the filepath of our new JDK. For Windows, we edit a PATH variable to include the file path of JDK.

### For Mac

Open up your Terminal. Check whether the language at the top is "bash" or "zsh"

![image](../_images/command-line-language.png)

 - If the top of your command prompt shell says "bash"....
	-  You can run the following on the command line to set up the JAVA_HOME environment variable. The line of code adds your JAVA_HOME variable to a file called “bash_profile”, which is where environment variables are stored. `!echo "export JAVA_HOME=$(/usr/libexec/java_home)" >> ~/.bash_profile`
	-  Then, to immediately update your "bash profile", run: `source ~/.bash_profile`
	-  Then, to test whether Java correctly installed, type `javac`. If you get a list of options, you correctly installed it. If your command line says "Not recognized," then it is not correctly installed.
-  If the top of your command prompt shell says "zsh"...
	-  You can run the following on the command line to set up the JAVA_HOME environment variable. The line of code adds your JAVA_HOME variable to a file called “zshrc”, which is where environment variables are stored. `echo "export JAVA_HOME=$(/usr/libexec/java_home)" >> ~/.zshrc`
	-  Then, to immediately update your "zshrc", run: `source ~/.zshrc`
	-  Then, to test whether Java correctly installed, type `javac`. If you get a list of options, you correctly installed it. If your command line says "Not recognized," then it is not correctly installed.



##  To  install Mallet
To download MALLET, click the following [link](http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip) or find the link on the [MALLET home page.](https://mimno.github.io/Mallet/) Once the zip file downloads, unzip it.

Once installed, verify that it's working

To test whether MALLET works on your computer, type in the file path for MALLET on the command line and import-file, like so:

- Mac:  `~/mallet-2.0.8/bin/mallet import-file` 
- Windows: `C:\mallet-2.0.8.\bin\mallet import-file`

If it’s working, then you’ll get a message that says “A tool for creating instance lists of feature vectors from comma-separated-values” and a list of options.

If you get a permissions error, type `chmod +x mallet-2.0.8/bin/mallet`

In [56]:
path_to_mallet = 'mallet-2.0.8/bin/mallet'

In [57]:
directory = f'{name_of_quote}_contexts/'

In [58]:
files = glob.glob(f"{directory}/*.txt")

In [59]:
files

['Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts/Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-1.txt',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts/Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-13.txt',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts/Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-12.txt',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts/Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-2.txt',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts/Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-10.txt',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts/Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-11.txt',
 'Finale-las

In [60]:
training_data = []
for file in files:
    text = open(file, encoding='utf-8').read()
    processed_text = little_mallet_wrapper.process_string(text, numbers='remove')
    training_data.append(processed_text)

In [61]:
original_texts = []
for file in files:
    text = open(file, encoding='utf-8').read()
    original_texts.append(text)

In [62]:
context_titles = [Path(file).stem for file in files]
# The Path().stem function extract the filename without the .txt extension

In [63]:
context_titles

['Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-1',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-13',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-12',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-2',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-10',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-11',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-3',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-7',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-6',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-4',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-5',
 'Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-8',


In [64]:
little_mallet_wrapper.print_dataset_stats(training_data)

Number of Documents: 13
Mean Number of Words per Document: 80.5
Vocabulary Size: 746


In [65]:
num_topics = 5 # Change this number to change the number of topicsnum_topics = 10 # Change this number to change the number of topics

In [66]:
training_data = training_data

In [67]:
#Change to your desired output directory
output_directory_path = f'topic-model-output/{name_of_quote}'

#No need to change anything below here
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

path_to_training_data           = output_directory_path + '/training.txt'
path_to_formatted_training_data = output_directory_path + '/mallet.training'
path_to_model                   = output_directory_path + '/mallet.model.' + str(num_topics)
path_to_topic_keys              = output_directory_path + '/mallet.topic_keys.' + str(num_topics)
path_to_topic_distributions     = output_directory_path + '/mallet.topic_distributions.' + str(num_topics)
path_to_word_weights            = output_directory_path + '/mallet.word_weights.' + str(num_topics)
path_to_diagnostics             = output_directory_path + '/mallet.diagnostics.' + str(num_topics) + '.xml'



In [68]:
little_mallet_wrapper.import_data(path_to_mallet,
                path_to_training_data,
                path_to_formatted_training_data,
                training_data)

Importing data...
Complete


In [69]:
little_mallet_wrapper.train_topic_model(path_to_mallet,
                      path_to_formatted_training_data,
                      path_to_model,
                      path_to_topic_keys,
                      path_to_topic_distributions,
                      path_to_word_weights,
                      path_to_diagnostics,
                      num_topics)

Training topic model...


Mallet LDA: 5 topics, 3 topic bits, 111 topic mask
Data loaded.
max tokens: 88
total tokens: 1046
<10> LL/token: -9.10459
<20> LL/token: -9.03836
<30> LL/token: -9.04434
<40> LL/token: -9.00169

0	1	middlemarch novel hidden moral works death woolf clarissa mrs lost paradise lived human passage finale lighthouse mist dalloway diffusion amano 
1	1	one george good acts mary unvisited james appears eliot ritchie grave thought iii oppression rather tombs number party toward dwight 
2	1	letters deronda daniel point sympathy may effects influence incalculably read faith however spirit great two quiet efficacy gwendolen definition word 
3	1	eliot dorothea life world half historical women also points visible much achievement diffusive lives live ramsay survived secular part owing 
4	1	like macmillan marx signed worth magazine men past nature matter lindley study man subject dead sentimental century publication poetry cent 

<50> LL/token: -8.95224
<60> LL/token: -8.99435
<70> LL/token: -9.01471

Complete


[beta: 4.08171] 
<820> LL/token: -7.95884
[beta: 2.99029] 
<830> LL/token: -7.93535
[beta: 3.47243] 
<840> LL/token: -7.8994

0	1.27155	marx visible century women one sociological reflects lifted mist clarissa suggest allusions thought lost paradise aspects incorporation selective awarded evening 
1	1.78537	george deronda study mary acts daniel allowing result drama clarify distinction beacon sweep deathwish midst dramatize promote remarks contexts idealism 
2	5.79554	eliot middlemarch letters life novel george man worth works death nature sympathy like also one ritchie survived amano paradise iii 
3	2.08715	dorothea hidden historical diffusive life novel faith clarissa may secular effects faithfully much deronda daniel incalculably moral like finale one 
4	2.24548	eliot good world macmillan middlemarch matter mrs real women woolf passi individuals analysis constitutes informed commenting intentionally announcements framing perspective 

[beta: 2.74664] 
<850> LL/token: -8.018
[beta: 3

### Display Topics and Top Words

In [70]:
topics = little_mallet_wrapper.load_topic_keys(path_to_topic_keys)

for topic_number, topic in enumerate(topics):
    print(f"Topic {topic_number}\n\n{topic}\n")

Topic 0

['worth', 'drama', 'intentionally', 'dramatize', 'simulate', 'chronicled', 'jewish', 'idealism', 'points', 'spread', 'laid', 'clarissa', 'may', 'solution', 'successor', 'raphael', 'survey', 'standard', 'ran', 'broadside']

Topic 1

['letters', 'sympathy', 'amano', 'individuals', 'sociological', 'announcements', 'institute', 'idealist', 'riverside', 'small', 'seen', 'knew', 'concept', 'fully', 'die', 'innocence', 'entombed', 'entombs', 'sees', 'closer']

Topic 2

['eliot', 'dorothea', 'middlemarch', 'one', 'george', 'life', 'novel', 'good', 'like', 'world', 'works', 'half', 'women', 'daniel', 'moral', 'mrs', 'lost', 'paradise', 'lived', 'visible']

Topic 3

['eliot', 'macmillan', 'world', 'tone', 'publication', 'dwight', 'past', 'death', 'deronda', 'human', 'middlemarch', 'analysis', 'informed', 'generalizing', 'remain', 'three', 'reconfigures', 'clarify', 'sweep', 'feels']

Topic 4

['survived', 'study', 'spirit', 'subject', 'woolf', 'depict', 'writers', 'technical', 'announce

## Load Topic Distributions

In [71]:
topic_distributions = little_mallet_wrapper.load_topic_distributions(path_to_topic_distributions)

In [72]:
#Let's look at number 5
topic_distributions[5]

[0.055570834179807044,
 0.09708205432320946,
 0.5342353022955671,
 0.19005762888204392,
 0.12305418031937247]

In [73]:
context_to_check = f"{name_of_quote}_contexts-6"

chunk_number = context_titles.index(context_to_check)

print(f"Topic Distributions for {context_titles[chunk_number]}\n")

print("Chunk contents:\n")
print(open(f'{directory}/{context_to_check}.txt', 'r').read())
print('\n')
for topic_number, (topic, topic_distribution) in enumerate(zip(topics, topic_distributions[chunk_number])):
    print(f"Topic {topic_number} {topic[:20]} \nProbability: {round(topic_distribution, 3)}\n")

Topic Distributions for Finale-last-sentence-full-VS-journals-200-char-before-and-750-after_contexts-6

Chunk contents:

icine, in politics, on the railway, on the land—is a distinctive feature of both the novel and play, and Eliot's study of provincial life moves steadily and forcefully toward the conclusion that “the [...].” The plays ran in repertory (with a few all-day performances) across the holiday season to February 2014. The DailyTelegraph, Evening Standard, and the Times each awarded the series four stars. Lastly, in this brief selective survey, Dwight A. Lindley III has expanded prior research on incorporation of aspects from Paradise Lost into Middlemarch in “The Way George Eliot Read Paradise Lost: Raphael and the Temptation of Speculative Thought” (Christianity and Literature 62.iii [Spring 2013]: 355–67). Repeated allusions and quotations from Paradise Lost appear persistently enough to suggest a closer relationship between the two works than has been previously promoted

### Top context chunks

In [749]:
training_data_context_titles = dict(zip(training_data, context_titles))
training_data_original_text = dict(zip(training_data, original_texts))

In [750]:
def display_top_titles_per_topic(topic_number=0, number_of_documents=5):
    
    print(f"Topic {topic_number}\n\n{topics[topic_number]}\n")

    for probability, document in little_mallet_wrapper.get_top_docs(training_data, topic_distributions, topic_number, n=number_of_documents):
        print(round(probability, 4), training_data_context_titles[document] + "\n")
    return

In [751]:
display_top_titles_per_topic(topic_number=0, number_of_documents=7)

Topic 0

['century', 'history', 'narrative', 'novel', 'scott', 'george', 'eliot', 'chapter', 'text', 'women', 'new', 'dickens', 'author', 'eighteenth', 'fielding', 'narrator', 'may', 'like', 'one', 'oliver']

0.9999 Chapter15-ALL-fulltexts_contexts-97

0.9995 Chapter15-ALL-fulltexts_contexts-97

0.9988 Chapter15-ALL-fulltexts_contexts-97

0.734 Chapter15-ALL-fulltexts_contexts-61

0.731 Chapter15-ALL-fulltexts_contexts-61

0.5366 Chapter15-ALL-fulltexts_contexts-63

0.4485 Chapter15-ALL-fulltexts_contexts-60



In [752]:
display_top_titles_per_topic(topic_number=1, number_of_documents=5)

Topic 1

['eliot', 'novels', 'women', 'literary', 'novel', 'one', 'george', 'literature', 'truth', 'narrative', 'narrator', 'voice', 'authority', 'fiction', 'woman', 'work', 'see', 'mill', 'new', 'human']

0.9999 Chapter15-ALL-fulltexts_contexts-2

0.9997 Chapter15-ALL-fulltexts_contexts-2

0.9995 Chapter15-ALL-fulltexts_contexts-2

0.7542 Chapter15-ALL-fulltexts_contexts-50

0.7512 Chapter15-ALL-fulltexts_contexts-50



In [753]:
display_top_titles_per_topic(topic_number=2, number_of_documents=6)

Topic 2

['eliot', 'george', 'one', 'like', 'dorothea', 'world', 'self', 'novel', 'middlemarch', 'life', 'lydgate', 'must', 'even', 'deronda', 'reality', 'daniel', 'marriage', 'characters', 'lewes', 'gwendolen']

0.9999 Chapter15-ALL-fulltexts_contexts-91

0.9996 Chapter15-ALL-fulltexts_contexts-91

0.8854 Chapter15-ALL-fulltexts_contexts-22

0.8839 Chapter15-ALL-fulltexts_contexts-22

0.8094 Chapter15-ALL-fulltexts_contexts-85

0.8059 Chapter15-ALL-fulltexts_contexts-85



In [754]:
display_top_titles_per_topic(topic_number=3, number_of_documents=6)

Topic 3

['medical', 'see', 'care', 'case', 'health', 'value', 'research', 'one', 'monomania', 'patients', 'would', 'rationing', 'note', 'may', 'new', 'practice', 'professional', 'act', 'doctors', 'gambling']

0.9118 Chapter15-ALL-fulltexts_contexts-26

0.9089 Chapter15-ALL-fulltexts_contexts-20

0.9019 Chapter15-ALL-fulltexts_contexts-26

0.7058 Chapter15-ALL-fulltexts_contexts-99

0.6938 Chapter15-ALL-fulltexts_contexts-94

0.5562 Chapter15-ALL-fulltexts_contexts-89



In [755]:
display_top_titles_per_topic(topic_number=4, number_of_documents=6)

Topic 4

['der', 'und', 'die', 'von', 'des', 'wie', 'das', 'auch', 'für', 'als', 'objektivität', 'ist', 'einer', 'eine', 'auf', 'den', 'realismus', 'nicht', 'mit', 'anm']

0.9999 Chapter15-ALL-fulltexts_contexts-45

0.9999 Chapter15-ALL-fulltexts_contexts-45

0.0111 Chapter15-ALL-fulltexts_contexts-63

0.0 Chapter15-ALL-fulltexts_contexts-30

0.0 Chapter15-ALL-fulltexts_contexts-46

0.0 Chapter15-ALL-fulltexts_contexts-60



In [756]:
display_top_titles_per_topic(topic_number=5, number_of_documents=6)

Topic 5

['chapter', 'leaf', 'leaves', 'lydgate', 'fol', 'eliot', 'rosamond', 'original', 'manuscript', 'fred', 'first', 'new', 'one', 'george', 'bulstrode', 'middlemarch', 'two', 'material', 'vincy', 'mary']

0.9999 Chapter15-ALL-fulltexts_contexts-8

0.9999 Chapter15-ALL-fulltexts_contexts-8

0.9999 Chapter15-ALL-fulltexts_contexts-8

0.9999 Chapter15-ALL-fulltexts_contexts-8

0.9999 Chapter15-ALL-fulltexts_contexts-8

0.9999 Chapter15-ALL-fulltexts_contexts-8



In [757]:
display_top_titles_per_topic(topic_number=6, number_of_documents=6)

Topic 6

['eliot', 'middlemarch', 'bichat', 'lydgate', 'death', 'casaubon', 'realism', 'text', 'body', 'tissues', 'one', 'chapter', 'novel', 'sense', 'discourse', 'george', 'foucault', 'century', 'medicine', 'life']

0.9999 Chapter15-ALL-fulltexts_contexts-33

0.9999 Chapter15-ALL-fulltexts_contexts-33

0.9999 Chapter15-ALL-fulltexts_contexts-33

0.9999 Chapter15-ALL-fulltexts_contexts-33

0.9999 Chapter15-ALL-fulltexts_contexts-33

0.9999 Chapter15-ALL-fulltexts_contexts-33



In [758]:
display_top_titles_per_topic(topic_number=7, number_of_documents=6)

Topic 7

['eliot', 'woolf', 'life', 'history', 'common', 'women', 'great', 'like', 'woman', 'one', 'historical', 'yet', 'though', 'men', 'may', 'detail', 'human', 'fiction', 'mrs', 'must']

1.0 Chapter15-ALL-fulltexts_contexts-79

1.0 Chapter15-ALL-fulltexts_contexts-79

1.0 Chapter15-ALL-fulltexts_contexts-79

1.0 Chapter15-ALL-fulltexts_contexts-79

1.0 Chapter15-ALL-fulltexts_contexts-79

1.0 Chapter15-ALL-fulltexts_contexts-79



In [759]:
display_top_titles_per_topic(topic_number=8, number_of_documents=6)

Topic 8

['middlemarch', 'eliot', 'george', 'lewes', 'scientific', 'lydgate', 'microscope', 'microscopy', 'century', 'farebrother', 'narrator', 'see', 'also', 'nineteenth', 'one', 'research', 'life', 'chapter', 'science', 'carpenter']

0.9999 Chapter15-ALL-fulltexts_contexts-52

0.9999 Chapter15-ALL-fulltexts_contexts-52

0.9999 Chapter15-ALL-fulltexts_contexts-52

0.9999 Chapter15-ALL-fulltexts_contexts-52

0.9997 Chapter15-ALL-fulltexts_contexts-52

0.9996 Chapter15-ALL-fulltexts_contexts-68



In [760]:
display_top_titles_per_topic(topic_number=9, number_of_documents=6)

Topic 9

['language', 'whewell', 'new', 'eliot', 'science', 'mind', 'george', 'ideas', 'discovery', 'history', 'like', 'life', 'great', 'natural', 'man', 'cambridge', 'consciousness', 'also', 'thought', 'time']

0.9999 Chapter15-ALL-fulltexts_contexts-84

0.9998 Chapter15-ALL-fulltexts_contexts-84

0.9995 Chapter15-ALL-fulltexts_contexts-84

0.9991 Chapter15-ALL-fulltexts_contexts-84

0.4137 Chapter15-ALL-fulltexts_contexts-31

0.2071 Chapter15-ALL-fulltexts_contexts-24

